# Декораторы

In [1]:
def decor(func):
    def inner():
        print('running inner()')
    return inner

@decor
def target():
    print('running target')

target()

running inner()


In [2]:
target

<function __main__.decor.<locals>.inner()>

In [3]:
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()  # <9>

running register(<function f1 at 0x7f5913f32b00>)
running register(<function f2 at 0x7f5913f335b0>)
running main()
registry -> [<function f1 at 0x7f5913f32b00>, <function f2 at 0x7f5913f335b0>]
running f1()
running f2()
running f3()


## Если registration.py импортируется и не запускается, то вывод будет выглядеть так:
* running register(<function f1 at 0x7f9a54753a30>) 
* running register(<function f2 at 0x7f9a54753be0>)

In [4]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [5]:
registry[1]

<function __main__.f2()>

In [6]:
registry[0]()

running f1()


# Реализация простого декоратора

In [7]:
import time

def clock(func):
    def clocked(*args): # 1
        t0 = time.perf_counter()
        result = func(*args) # 2
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked # 3

* 1 Определяем внутреннюю фун-ию ```clocked```, принимающую произвольное число позиционных аргументов.
* 2 Эта функция работает только потому, что замыкание ```clocked``` включает свободную переменную func.
* 3 Возвращает внутрненюю фун-ию взамен декорирующей. 

In [8]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12313016s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000038s] factorial(1) -> 1
[0.00001640s] factorial(2) -> 2
[0.00002985s] factorial(3) -> 6
[0.00004283s] factorial(4) -> 24
[0.00005589s] factorial(5) -> 120
[0.00006938s] factorial(6) -> 720
6! = 720


Код выше эквивалентен:

```python
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)
```
тоже самое что и

```python
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)
    factorial = clock(factorial)
```

То есть в обоих случиях декоратор ```@clock``` получает фун-ию ```factorial``` в качестве аргумента func. Затем он создает и возвращает функцию ```clocked()```, которую интерпритатор Python за кулисами связывает с именем factorial.
На самом деле, если вывести атрибут ```__name__```

## Улучшенный декоратор clock
Декоратор выше имеет ряд недостатоков: он не поддерживает именованные аргументы и маскирует атрибуты name и doc декорированной фун-ей

In [9]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked # 3

In [10]:
@clock
def fibonachi(n):
    if n < 2:
        return n
    return fibonachi(n-2) + fibonachi(n-1)

if __name__ == '__main__':
    print(fibonachi(6))

[0.00000024s] fibonachi(0) -> 0
[0.00000048s] fibonachi(1) -> 1
[0.00010180s] fibonachi(2) -> 1
[0.00000024s] fibonachi(1) -> 1
[0.00000024s] fibonachi(0) -> 0
[0.00000024s] fibonachi(1) -> 1
[0.00002170s] fibonachi(2) -> 1
[0.00004268s] fibonachi(3) -> 2
[0.00016642s] fibonachi(4) -> 3
[0.00000000s] fibonachi(1) -> 1
[0.00000024s] fibonachi(0) -> 0
[0.00000024s] fibonachi(1) -> 1
[0.00002122s] fibonachi(2) -> 1
[0.00004220s] fibonachi(3) -> 2
[0.00000024s] fibonachi(0) -> 0
[0.00000024s] fibonachi(1) -> 1
[0.00002098s] fibonachi(2) -> 1
[0.00000024s] fibonachi(1) -> 1
[0.00000024s] fibonachi(0) -> 0
[0.00000048s] fibonachi(1) -> 1
[0.00002432s] fibonachi(2) -> 1
[0.00010657s] fibonachi(3) -> 2
[0.00014830s] fibonachi(4) -> 3
[0.00021148s] fibonachi(5) -> 5
[0.00039959s] fibonachi(6) -> 8
8


Непроизводительные затраты бросаются в глаза: fibonachi(1) -> 7 вызовов, fibonachi(2) -> 5 вызовов и тд. *Что делать?*

## Кэширование с помощью functools.lru_cache

Этот декоратор очень полезен, когда смысл оптимизации заключается в сохранении предидущих дорогостоющих вызовов фун-ии, что позволяет избежать повторного вычисления с теми же аргументами.

* 1 ```lru_cache``` следует вызыват ькак обычную фун-ию. Причина в том что декоратор принимает конфигурационные параметры
* 2 пример композиции декораторов ```lru_cache()``` применяется фун-ии возвращенной декоратором ```@clock```

In [12]:
@functools.lru_cache() # 1
@clock # 2
def fibonachi_2(n):
    if n < 2:
        return n
    return fibonachi_2(n-2) + fibonachi_2(n-1)

if __name__ == '__main__':
    print(fibonachi_2(6))

[0.00000024s] fibonachi_2(0) -> 0
[0.00000048s] fibonachi_2(1) -> 1
[0.00013375s] fibonachi_2(2) -> 1
[0.00000048s] fibonachi_2(3) -> 2
[0.00016022s] fibonachi_2(4) -> 3
[0.00000072s] fibonachi_2(5) -> 5
[0.00018573s] fibonachi_2(6) -> 8
8


In [21]:
fibonachi_2(40)

102334155

In [19]:
def fibonachi_non_text(n):
    if n < 2:
        return n
    return fibonachi_non_text(n-2) + fibonachi_non_text(n-1)

fibonachi_non_text(40)

102334155

Разница во времени выполенения коллосальная 0.1 сек против 30 сек

```lru_cache``` можно настроить

```python
functools.lru_cache(maxsize=128, typed=False)
```
* maxsize - Сколько результатов хранить
* typed - Если True, то результы для разных типов хранятся поразень

## Одиночная диспетчирезация и обобщённые функции

### Одиночная

In [22]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [23]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [24]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [25]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


### Обобщенная

In [26]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # <2>
def _(text):            # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [27]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [28]:
htmlize(abs)


'<pre>&lt;built-in function abs&gt;</pre>'

In [31]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


* 1 ```@singledispatch``` помечает базовую функцию, которая обрабатывает тип object
* 2 Каждая специализированная фун-ия снабжается декоратором ```@base_object```
* 3 Имена специализированных фун-ий не существенны, поэтому _ в качестве имени
* 4 Для каждого типа, нуждающегося в специальнной обработке, регистрируется новая функция ```numbers.Integral``` - виртуальный класс ```int```
* 5 Можно указывать несколько декораторов ```register```, если требуется, чтобы одна функция поддерживала несколько типов   

## Композиция декораторов

Когда два ```@d1``` и ```@d2``` декоратора применяются к одной фун-ции это тоже самое что и в результате композиции ```f = d1(d1(f))```
Иными словами:
```
@d1
@d2
def f():
    print(f)
```
Эвивалентен:
```
def f():
    print(f)

f = d1(d2(f))
```

## Параметризированные декораторы

Т.к декоратор в качетсве первого аргумента функции-декоратору, берет декорируемую функцию, какже передать другие аргументы?
Ответ: написать *фабрику декораторов*, которая бы принимает аргументы и возвращает декоратор, который затем применяется к декорируемой функции

In [32]:
registry = [] 

def register(func): 
    print('running register(%s)' % func) 
    registry.append(func) 
    return func 

@register 
def f1():
    print('running f1()')

running register(<function f1 at 0x7f5913f330a0>)


In [33]:
print('run main()')
print('registry ->', registry)
f1()

run main()
registry -> [<function f1 at 0x7f5913f330a0>]
running f1()


## Параметризированный регистрационный декоратор

In [39]:
registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')


running register(active=False)->decorate(<function f1 at 0x7f58f98a5510>)
running register(active=True)->decorate(<function f2 at 0x7f58f98a5360>)


* 1 Теперь ```registry``` имет тип ```set```, чтобы ускорить добавление и удаление функций
* 2 Функция ```register``` принимает необязательный именованный аргумент.
* 3 Собственно декоратором является собственная функция ```decorate```, она принимает в качетве аргумента функцию
* 4 Регистриуем ```func```, только если аргумент ```active``` (определенный в замыкании) равен ```True```
* 5 Если ```not active``` и функция ```func``` присутсвует в ```registry```, удаляем ее
* 6 Поскольку ```decorate``` - декоратор, он ддолжен возвращать функцию
* 7 Функция ```register``` - наша *фабрика декораторов*, поэтому возвращает ```decorate```
* 8 Фабрику ```@register``` следует вызвать как функцию, передавая ей нужные параметры
* 9 Даже если параметров нет, ```register``` все равно нужно вызвать как функцию - ```@register()``` - чтобы она вернула настоящий декоратор ```decorate```

In [44]:
registry # <1>

{<function __main__.f2()>}

In [45]:
register()(f3) # <2>

running register(active=True)->decorate(<function f3 at 0x7f5913f31000>)


<function __main__.f3()>

In [46]:
registry # <3>

{<function __main__.f2()>, <function __main__.f3()>}

In [47]:
register(active=False)(f2) # <4>

running register(active=False)->decorate(<function f2 at 0x7f58f98a5360>)


<function __main__.f2()>

In [48]:
registry # <5>

{<function __main__.f3()>}

* 1 После импортирования модуля ```f2``` оказывается в ```registry``` 
* 2 Выражение ```register()``` возвращает декоратор ```decorate```, который затем применяется а ```f3```
* 3 В предидущей строке ```f3``` была добавлена в ```registery```
* 4 Этот вызов удаляет ```f2``` из ```registery```
* 5 Убеждаемся, что ```f3``` осталась в ```registery```

## Параметризованный декоратор clock

In [50]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # <1>
    def decorate(func):      # <2>
        def clocked(*_args): # <3>
            t0 = time.time()
            _result = func(*_args)  # <4>
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # <6>
            print(fmt.format(**locals()))  # <7>
            return _result  # <8>
        return clocked  # <9>
    return decorate  # <10>

if __name__ == '__main__':

    @clock()  # <11>
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12313986s] snooze(0.123) -> None
[0.12313557s] snooze(0.123) -> None
[0.12313676s] snooze(0.123) -> None


* 1 Теперь ```clock``` - наша фабрика параметризованных декораторов
* 2 ```decorate``` - это собственно конструктор
* 3 ```clocked``` обертывает декорированную функцию
* 4 ```_result_``` - результат, возвращенный декорированной функцией
* 5 В ```_args``` хранятся фактические аргументы ```clocked```, тогда как ```args``` - отображаемая строка
* 6 ```result``` - строковое представление ```_result```, предназначенное для отображения
* 7 Использование ```**locals()``` позволяет ссылаться в ```fmt``` на любую локальную переменную ```clocked```
* 8 ```clocked``` заменяет декорированную функцию без аргументов, поэтом должна возвращать то, что вернула бы эта функция в отсутсвие декоратора
* 9 ```decorate``` возвращает ```clocked```
* 10 ```clock``` возвращает ```decorate```
* 11 В этом тесте ```clock()``` вызывается без аргументов, поэтому декоратор будет использовать, форматную строку по умолчанию